# LDA Model for Visualization

In [0]:
# imports needed for data
import pandas as pd
import pickle

In [0]:
# read in the data with pandas, this model uses a pickle file
data = pd.read_pickle(DATA HERE.pkl)
print(data.shape)
data.head()

After running head, this should be the DTM (Document Term Matrix)
Every row would be a document, or review and the columns would be the tokens / words

In [0]:
# imports for LDA with Gensim
from gensim import matutils, models
import scipy.sparse

In [0]:
# one of the required inputs for this LDA model is a TDM, 

tdm = data.transpose()
tdm.head()

the rows are the tokens / words and the columns are the documents

In [0]:
# we're going to put the TDM into a new gensim format

sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [0]:
# gensim also requires a dictionary of all the terms, and possibly their location.

cv = pickle.load(open("SOMETHING.pkl", "rb"))
id2word = dic((v, k) for k, v in cv.vocabulary_.items())

now that we have the corpus (TDM) and id2word (dictionary of location: term) we will need to specify 2 other parameters - The nunber of Topics and The number of Passes. We'll start the number of topics at 2, see if it makes sense and adjust form there

In [0]:
# set the lda model and the parameters
# 2 topics
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)

In [0]:
# 3 topics
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)

In [0]:
# 4 topics
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)

The output: first row shows the top words for the 1st topic, then below will be the rows for the 2nd topic, etc


The next level will be to get Nouns and Adjectives only. This will polish the topics being found. 

In [0]:
# Let's create a function to pull out the nouns and adj from the text.
# NN is used for nouns and JJ is used for Adjectives

def nouns_adj(text):
  is_noun_adj = lambda pos: pos[:2] = 'NN' or pos[:2] == 'JJ'
  tokenized = TOKENIZED TEXT FROM DB
  nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj]
  return ' '.join(nouns_adj)

In [0]:
# apply the nouns adj function  to the transcripts to filter

data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

the output will be each doc with their transcript

In [0]:
# create a new DTM only using the nouns and adj

data_cv = data_nouns_adj.transcript
data_dtm = pd.DataFrame(data_cv.toarray(), columns = data_cv.get_feature_names)  
data_dtm.index = data_nouns_adj.index
data_dtm

now we can recreate everything to include what we've made


In [0]:
# create the gensim corpus

corpusna = matutils.Sparse2Corpus(scipy.sparse,scr_matrix(data_dtm.transpose()))

# create the vocabulary dictionary

id2wordna = dict((v, k) for k, v in  data_cv.vocabulary_.items())

In [0]:
# start with 2 topics again

ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldna.print_topics()

In [0]:
# try 3 topics

ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldna.print_topics()

In [0]:
# try 4 topics

ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldna.print_topics()

When the topics start looking different we can go with that to the next step.

In [0]:
# run more iterations on our "final model"
# what increasing the passes does is it stabalizes which words falls into a topic

ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldna.print_topics()

In [0]:
# now we can look at which topic each doc or transcript contains 

corpus_transformed = ldna[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtm.index))